In [ ]:
import numpy as np
from TMDP import TMDP
from River_swim import River

from algorithms import *
from model_functions import *

import matplotlib.pyplot as plt
import gymnasium as gym
from FrozenLake import *
from PG_algorithms import *
from ActorCritic import *
from ReplayBuffer import *
#np.set_printoptions(precision=4)
import math
from utils import *

nrows = 10
nS = nrows**2
nA = 4
seed = get_current_seed()
gamma = .99
tau = 1.
#tmdp = TMDP(river, xi, tau=tau, gamma=gamma, seed=seed)
#mdp = TMDP(river, xi, tau=0., gamma=gamma, seed=seed)

env = FrozenLakeEnv(is_slippery=False, seed=seed, desc=generate_random_map(nrows), r_shape=True)#, render_mode="human")
xi_frozen = np.ones(env.nS) * 1/env.nS
tmdp = TMDP(env, xi_frozen, tau=tau, gamma=gamma, seed=seed)
tmdp.reset()

In [ ]:
rep_buffer = ReplayBuffer(max_size=int(1e6), input_shape=(1,), seed=seed)

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
ref_policy = ActorNet(nS, nA, hidden_dim=64).to(device)
policy_pi = ActorNet(nS, nA, hidden_dim=64).to(device)

q1_func = QNet(nS, nA, hidden_dim=64).to(device)
q2_func = QNet(nS, nA, hidden_dim=64).to(device)
q1_target = QNet(nS, nA, hidden_dim=64).to(device)
q2_target = QNet(nS, nA, hidden_dim=64).to(device)

In [ ]:
ref_opt = torch.optim.AdamW(ref_policy.parameters(), lr=1e-3)
q1_opt = torch.optim.AdamW(q1_func.parameters(), lr=1e-3)
q2_opt = torch.optim.AdamW(q2_func.parameters(), lr=1e-3)


In [ ]:
#env.render_mode = "human"

In [ ]:
tmdp.update_tau(.5)
tmdp.reset()
cur_res = curriculum_SAC(tmdp, policy_pi, ref_policy, q1_target, q2_target, q1_func,
                           q2_func, ref_opt, q1_opt, q2_opt, rep_buffer, 
                           alpha=.2, alpha_u=.2, beta=0.05, episodes=2000000, 
                           batch_size=256, sample_steps=8, update_rate=32, 
                           biased=False)

In [ ]:
print(q1_func.get_values())

In [ ]:
pi = get_policy(q1_func.get_values())
tmdp = TMDP(env, xi_frozen, tau=0., gamma=gamma, seed=seed)
tmdp.update_tau(0.)
env.render_mode = "human"
tmdp.reset()
done = False
step = 0
while True:
    a = greedy(tmdp.env.s, pi, tmdp.env.allowed_actions[int(tmdp.env.s)])
    s_prime, reward, flags, prob = tmdp.step(a)
    if flags["done"]:
        tmdp.reset()
    step +=1
    if step > max(100,nrows*2):
        break